<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 14:26:05
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 94.66 K (0.68%)
Current PnL: -21.18 L (-14.11%)
CY Booked + Current PnL: -9.28 L (-6.18%)
-------------------
Total profit:  1.83 L
Total loss:  -23.02 L
-------------------
Total Booked + Current PnL: 17.70 L (14.37%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.48%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.37 L (62.99%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.12,56.0,X-LC,3.71,213542.0,9042.0,14243.0,0.41,4.42,6.67,11.39,10.0,0.63,1.53,13.77,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,95.73,47.0,M-SC,1.67,84331.0,-16446.0,16537.0,0.05,-16.32,19.61,0.09,245.0,-0.99,0.61,10.79,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.16,58.0,X-MC,1.26,212205.0,20463.0,17295.0,0.18,10.67,8.15,19.69,99.0,1.18,1.52,18.20,XY25,NTT,AC
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.07,148807.0,12674.0,18452.0,-0.47,9.31,12.40,22.86,91.0,0.69,1.07,23.10,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-13.29,55.0,X-LC,2.87,217136.0,6770.0,22018.0,1.28,3.22,10.14,13.68,37.0,0.31,1.56,20.27,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.0,-33.65,70.0,M-SC,12.25,82626.0,-18533.0,93425.0,4.40,-18.32,113.07,74.03,202.0,-0.20,0.59,16.51,AR,BTT,AUTO
49,NESTLEIND,1377.0,-3.13,71.0,X-LC,2.60,297952.0,32526.0,24253.0,4.25,12.25,8.14,21.40,11.0,1.34,2.14,19.71,XY25,NTT,FMCG
84,WHIRLPOOL,2270.0,-45.34,50.0,M-SC,4.24,92498.0,1000.0,77754.0,4.25,1.09,84.06,86.07,223.0,0.01,0.66,34.32,XR,NTT,DURABLES
63,SHALBY,327.0,1193.46,57.0,M-SC,23.04,175883.0,-5260.0,50725.0,3.66,-2.90,28.84,25.10,235.0,-0.10,1.26,40.95,XY24,NTT,HEALTHCARE
57,REPCOHOME,880.0,-56.78,65.0,H-SC,2.71,247107.0,-39398.0,296726.0,3.56,-13.75,120.08,89.82,134.0,-0.13,1.78,30.74,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.40,856.52,36.0,H-SC,5.20,82260.0,-26460.0,106140.0,-1.79,-24.34,129.03,73.29,161.0,-0.25,0.59,10.03,XR,NTT,ENTERTAINMENT
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.74,80736.0,-32813.0,72864.0,-1.31,-28.90,90.25,35.27,268.0,-0.45,0.58,97.60,XR,NTT,HOTELS
31,HINDZINC,730.22,35.55,66.0,M-LC,9.47,220915.0,15839.0,96739.0,-1.04,7.72,43.79,54.89,52.0,0.16,1.59,33.68,X5K,ATH,METALS
10,BANDHANBNK,400.00,-13.31,46.0,H-SC,2.90,216464.0,-62095.0,319544.0,-1.02,-22.29,147.62,92.42,151.0,-0.19,1.55,25.10,XY24,NTT,BANKS
50,PGHH,17907.65,-26.71,57.0,X-MC,3.25,212025.0,11205.0,56589.0,-1.00,5.58,26.69,33.76,80.0,0.20,1.52,11.07,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,69.0,M-SC,17.63,149940.0,720.0,143463.0,0.98,0.48,95.68,96.62,208.0,0.01,1.08,62.27,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,11.56,131139.0,-11340.0,30857.0,2.08,-7.96,23.53,13.70,163.0,-0.37,0.94,54.78,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,95.73,47.0,M-SC,1.67,84331.0,-16446.0,16537.0,0.05,-16.32,19.61,0.09,245.0,-0.99,0.61,10.79,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,1.90,220509.0,-49158.0,85491.0,-0.26,-18.23,38.77,13.47,138.0,-0.58,1.58,12.12,XY24,NTT,PAINTS
17,CERA,9475.0,-23.87,40.0,H-SC,2.22,138621.0,-37282.0,79305.0,-0.43,-21.19,57.21,23.89,149.0,-0.47,1.00,19.72,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.21,45.0,H-MC,7.15,102534.0,-29301.0,71528.0,0.54,-22.23,69.76,32.03,98.0,-0.41,0.74,14.85,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.14,207900.0,4730.0,37401.0,0.59,2.33,17.99,20.74,55.0,0.13,1.49,8.00,XY25,NTT,FINANCE
59,SAIL,228.00,44.35,52.0,M-MC,12.27,228148.0,3186.0,164472.0,1.69,1.42,72.09,74.53,192.0,0.02,1.64,34.13,XY24,BTT,STEEL
31,HINDZINC,730.22,35.55,66.0,M-LC,9.47,220915.0,15839.0,96739.0,-1.04,7.72,43.79,54.89,52.0,0.16,1.59,33.68,X5K,ATH,METALS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.76,155130.0,4185.0,105271.0,0.72,2.77,67.86,72.51,53.0,0.04,1.11,8.96,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,55.0,M-LC,8.73,157884.0,17044.0,27440.0,-0.42,12.10,17.38,31.59,66.0,0.62,1.13,41.80,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,35.55,66.0,M-LC,9.47,220915.0,15839.0,96739.0,-1.04,7.72,43.79,54.89,52.0,0.16,1.59,33.68,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,61.0,H-MC,13.02,193373.0,13565.0,98852.0,-0.48,7.54,51.12,62.52,88.0,0.14,1.39,44.20,XR,NTT,BANKS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.76,155130.0,4185.0,105271.0,0.72,2.77,67.86,72.51,53.0,0.04,1.11,8.96,XR,NTT,IT
84,WHIRLPOOL,2270.00,-45.34,50.0,M-SC,4.24,92498.0,1000.0,77754.0,4.25,1.09,84.06,86.07,223.0,0.01,0.66,34.32,XR,NTT,DURABLES
36,INDIAMART,4810.62,-55.20,41.0,H-SC,8.08,124948.0,1612.0,130021.0,1.19,1.31,104.06,106.72,119.0,0.01,0.90,24.99,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144202.0,-132321.0,242389.0,1.64,-47.85,168.09,39.81,54.0,-0.55,1.04,1.64,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52122.0,-12884.0,163814.0,-0.13,-19.82,314.29,232.18,198.0,-0.08,0.37,1.64,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.31,38.0,X-MC,1.84,229616.0,-18115.0,93178.0,0.53,-7.31,40.58,30.30,92.0,-0.19,1.65,5.01,X40,ATH,ELECTRICAL
34,ICICIPRULI,790.00,-22.94,40.0,X-MC,2.20,132560.0,-3228.0,45985.0,-0.25,-2.38,34.69,31.48,107.0,-0.07,0.95,9.45,X40,ATH,INSURANCE
12,BATAINDIA,2096.00,-36.44,41.0,X-SC,4.33,94985.0,-33685.0,78980.0,1.84,-26.18,83.15,35.20,219.0,-0.43,0.68,9.02,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.04,44.0,H-LC,0.61,155820.0,-30275.0,77754.0,-0.07,-16.27,49.90,25.51,15.0,-0.39,1.12,14.83,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.16,58.0,X-MC,1.26,212205.0,20463.0,17295.0,0.18,10.67,8.15,19.69,99.0,1.18,1.52,18.20,XY25,NTT,AC
28,HAVELLS,2069.16,-7.31,38.0,X-MC,1.84,229616.0,-18115.0,93178.0,0.53,-7.31,40.58,30.30,92.0,-0.19,1.65,5.01,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.07,148807.0,12674.0,18452.0,-0.47,9.31,12.40,22.86,91.0,0.69,1.07,23.10,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-3.13,71.0,X-LC,2.60,297952.0,32526.0,24253.0,4.25,12.25,8.14,21.40,11.0,1.34,2.14,19.71,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52122.0,-12884.0,163814.0,-0.13,-19.82,314.29,232.18,198.0,-0.08,0.37,1.64,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144202.0,-132321.0,242389.0,1.64,-47.85,168.09,39.81,54.0,-0.55,1.04,1.64,XY24,NTT,AUTO
73,TCS,4389.96,-29.97,45.0,X-LC,12.48,279048.0,-66908.0,133608.0,-0.04,-19.34,47.88,19.28,1.0,-0.50,2.00,3.16,X40,ATH,IT
21,COLPAL,3767.14,-4.54,42.0,X-MC,7.75,222710.0,-40655.0,154004.0,1.12,-15.44,69.15,43.04,84.0,-0.26,1.60,3.39,XY25,ATH,FMCG
9,BAJAJHFL,181.50,-14.48,44.0,X-MC,5.81,180203.0,-20303.0,117456.0,-0.23,-10.13,65.18,48.45,90.0,-0.17,1.29,3.92,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52122.0,-12884.0,163814.0,-0.13,-19.82,314.29,232.18,198.0,-0.08,0.37,1.64,XY24,NTT,MISC
55,RELAXO,1176.00,-39.01,45.0,X-SC,3.98,83096.0,-62064.0,137989.0,0.00,-42.76,166.06,52.31,136.0,-0.45,0.60,11.23,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.44,41.0,X-SC,4.33,94985.0,-33685.0,78980.0,1.84,-26.18,83.15,35.20,219.0,-0.43,0.68,9.02,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.61,48.0,X-SC,1.01,107341.0,5337.0,38589.0,-0.07,5.23,35.95,43.06,122.0,0.14,0.77,23.24,X40N,ATH,MISC
32,HONAUT,58357.33,-23.12,45.0,X-SC,2.40,108315.0,-19623.0,66755.0,-0.14,-15.34,61.63,36.84,143.0,-0.29,0.78,10.89,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-29.97,45.0,X-LC,12.48,279048.0,-66908.0,133608.0,-0.04,-19.34,47.88,19.28,1.0,-0.50,2.00,3.16,X40,ATH,IT
39,INFY,2275.00,-19.60,47.0,X-LC,7.54,313621.0,733.0,170955.0,-0.14,0.23,54.51,54.87,3.0,0.00,2.25,6.90,X40,BTT,IT
41,ITC,452.00,-38.77,51.0,X-LC,2.88,197849.0,-2289.0,23188.0,1.18,-1.14,11.72,10.44,4.0,-0.10,1.42,4.36,X40,NTT,FMCG
82,VBL,671.64,-18.39,53.0,X-LC,10.58,294118.0,-21724.0,134383.0,3.51,-6.88,45.69,35.67,5.0,-0.16,2.11,6.02,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,51.0,H-LC,7.24,250104.0,-11515.0,130729.0,0.72,-4.40,52.27,45.57,7.0,-0.09,1.80,6.42,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6655.56,54.0,L-SC,6.14,74118.0,-19692.0,96309.0,0.66,-20.99,129.94,81.67,269.0,-0.20,0.53,43.53,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.74,80736.0,-32813.0,72864.0,-1.31,-28.90,90.25,35.27,268.0,-0.45,0.58,97.60,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,43.0,H-SC,9.47,90465.0,-7515.0,29121.0,0.15,-7.67,32.19,22.05,152.0,-0.26,0.65,31.50,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,11.56,131139.0,-11340.0,30857.0,2.08,-7.96,23.53,13.70,163.0,-0.37,0.94,54.78,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,52.0,H-SC,10.25,143922.0,-35149.0,100875.0,-0.68,-19.63,70.09,36.71,135.0,-0.35,1.03,42.80,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.63,149940.0,720.0,143463.0,0.98,0.48,95.68,96.62,208.0,0.01,1.08,62.27,XY25,NTT,FINANCE
4,ANGELONE,3033.00,15.31,69.0,X-SC,9.36,204237.0,13231.0,44462.0,1.86,6.93,21.77,30.21,157.0,0.30,1.47,29.25,X40N,NTT,FINANCE
63,SHALBY,327.00,1193.46,57.0,M-SC,23.04,175883.0,-5260.0,50725.0,3.66,-2.90,28.84,25.10,235.0,-0.10,1.26,40.95,XY24,NTT,HEALTHCARE
68,SURYODAY,240.00,57.89,52.0,H-SC,10.25,143922.0,-35149.0,100875.0,-0.68,-19.63,70.09,36.71,135.0,-0.35,1.03,42.80,XR,NTT,BANKS
31,HINDZINC,730.22,35.55,66.0,M-LC,9.47,220915.0,15839.0,96739.0,-1.04,7.72,43.79,54.89,52.0,0.16,1.59,33.68,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,44.12
2,50,75.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.33
LC    33.80
MC    21.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.61
X40      14.75
XY25     12.26
XR       10.92
X40N     10.23
AR        8.20
OX40N     7.61
X200      1.82
X5K       1.59
SR        1.12
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.48
X-LC    22.71
X-MC    16.06
M-SC    12.73
M-LC     5.32
H-LC     4.74
X-SC     4.67
H-MC     3.81
M-MC     1.64
L-SC     1.45
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.20,-3.81,38.02
IT,12.93,-16.65,79.74
FINANCE,12.24,-10.91,61.18
MISC,6.86,-20.24,86.87
BANKS,6.34,-12.77,72.09
PAINTS,5.54,-19.63,37.46
ELECTRICAL,5.35,-10.69,49.53
INSURANCE,3.94,-0.92,38.65
AUTO,2.86,-44.37,113.38


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3309670.0,22
XR,1387828.0,14
AR,1189459.0,9
X40,782703.0,10
X40N,603163.0,8
OX40N,586683.0,10
XY25,475710.0,8
SR,264428.0,2
X5K,96739.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3452247.0,25
M-SC,1266758.0,15
X-MC,1090031.0,11
X-LC,1076081.0,13
X-SC,530589.0,6
H-MC,350071.0,3
H-LC,280093.0,3
M-LC,266851.0,4
L-SC,258569.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1196199.0      6
           XR         827156.0      7
           AR         820464.0      5
M-SC       XY24       685003.0      6
X-MC       XY24       587072.0      4
X-LC       X40        489519.0      5
           XY24       305979.0      2
X-SC       X40N       287795.0      4
H-SC       OX40N      274936.0      4
           SR         264428.0      2
X-MC       X40        214204.0      4
H-LC       AR         208483.0      2
X-LC       X40N       197912.0      3
H-MC       XY24       179691.0      1
X-MC       XY25       171299.0      2
L-SC       XR         169173.0      2
M-MC       XY24       164472.0      1
X-SC       XY24       163814.0      1
M-SC       AR         160512.0      2
           OX40N      150823.0      4
           XY25       143463.0      1
           XR         126957.0      2
X-MC       X40N       117456.0      1
M-LC       XR         105271.0      1
H-MC       XR          98852.0      1
M-LC       X5K         96739.0      1
L-SC       OX40N       89396.0      1
X-LC       XY25        82671.0      3
X-SC       X40         78980.0      1
H-LC       X200        71610.0      1
H-MC       OX40N       71528.0      1
H-SC       MH          69064.0      1
L-MC       XR          60419.0      1
L-LC       XY25        40876.0      1
M-LC       XY25        37401.0      1
           XY24        27440.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
